In [3]:
# --- 2_RPi_MediaPipe_Client.ipynb ---
# Run this on your Raspberry Pi 4B

import cv2
import mediapipe as mp
import numpy as np
import pickle
import time
import os
from IPython import display as ipydisplay

mp_pose = mp.solutions.pose

# ------------------ CONFIG ------------------
RTSP_URL = "rtsp://10.227.207.170:8554/webcam.sdp"   # ← Laptop stream
EXERCISE_NAME = "romanian_deadlift"                 # Change as needed
# -------------------------------------------

# Optimize OpenCV for RTSP
os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = (
    "rtsp_transport;tcp|"
    "fflags;nobuffer+discardcorrupt|"
    "flags;low_delay|"
    "max_delay;0|"
    "err_detect;ignore_err"
)

# Load reference pose sequence
pkl_path = f"{EXERCISE_NAME}.pkl"
if not os.path.exists(pkl_path):
    raise FileNotFoundError(f"Reference not found: {pkl_path}. Run extraction first!")

with open(pkl_path, 'rb') as f:
    reference_sequence = pickle.load(f)

print(f"Loaded {len(reference_sequence)} reference frames from {pkl_path}")

# Try GStreamer with hardware decode (recommended)
gst_pipeline = (
    f"rtspsrc location={RTSP_URL} latency=200 protocols=tcp ! "
    "rtph264depay ! h264parse ! v4l2h264dec ! "
    "videoconvert ! video/x-raw,format=BGR,width=640,height=480,framerate=15/1 ! "
    "appsink drop=1 max-buffers=1"
)

print("Connecting to stream...")
cap = cv2.VideoCapture(gst_pipeline, cv2.CAP_GSTREAMER)

# Fallback to FFmpeg
if not cap.isOpened():
    print("GStreamer failed → trying FFmpeg...")
    cap = cv2.VideoCapture(RTSP_URL, cv2.CAP_FFMPEG)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

if not cap.isOpened():
    raise ConnectionError(f"Cannot connect to {RTSP_URL}")

print("Connected! Starting live pose overlay...")

pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# IPython display
display_handle = ipydisplay.display(ipydisplay.Image(data=b''), display_id=True)

# Animation control
ref_idx = 0
anim_fps = 10
frame_delay = 1.0 / anim_fps
last_ref_time = time.time()
angle_thr = 20
frame_cnt = 0

def calculate_angle(a, b, c):
    a = np.array(a); b = np.array(b); c = np.array(c)
    ab = a - b; bc = c - b
    cosine = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine, -1.0, 1.0))
    return np.degrees(angle)

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            time.sleep(0.1)
            continue

        # Process every 3rd frame (~10 FPS)
        if frame_cnt % 3 != 0:
            frame_cnt += 1
            continue
        frame_cnt += 1

        h, w, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)

        canvas = np.zeros((h, w, 3), dtype=np.uint8)

        # --- Draw REFERENCE (BLUE) ---
        ref_lm = reference_sequence[ref_idx]
        for a, b in mp_pose.POSE_CONNECTIONS:
            s, e = ref_lm[a], ref_lm[b]
            if s[3] > 0.1 and e[3] > 0.1:
                cv2.line(canvas,
                         (int(s[0]*w), int(s[1]*h)),
                         (int(e[0]*w), int(e[1]*h)),
                         (255, 0, 0), 2)  # Blue

        # --- Draw USER (GREEN/RED) ---
        if results.pose_landmarks:
            user_lm = [(lm.x, lm.y, lm.z, lm.visibility) for lm in results.pose_landmarks.landmark]
            for a, b in mp_pose.POSE_CONNECTIONS:
                r1, r2 = ref_lm[a], ref_lm[b]
                u1, u2 = user_lm[a], user_lm[b]
                if all(x[3] > 0.1 for x in [r1, r2, u1, u2]):
                    # Find parent joint
                    parent = None
                    for conn in mp_pose.POSE_CONNECTIONS:
                        if conn[1] == b:
                            parent = conn[0]
                            break
                    if parent is not None:
                        rp = ref_lm[parent]
                        up = user_lm[parent]
                        if rp[3] > 0.1 and up[3] > 0.1:
                            ref_ang = calculate_angle((r1[0], r1[1]), (r2[0], r2[1]), (rp[0], rp[1]))
                            usr_ang = calculate_angle((u1[0], u1[1]), (u2[0], u2[1]), (up[0], up[1]))
                            color = (0, 255, 0) if abs(ref_ang - usr_ang) < angle_thr else (0, 0, 255)
                            cv2.line(canvas,
                                     (int(u1[0]*w), int(u1[1]*h)),
                                     (int(u2[0]*w), int(u2[1]*h)),
                                     color, 2)

        # --- Update display ---
        _, buf = cv2.imencode('.jpeg', canvas, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
        display_handle.update(ipydisplay.Image(data=buf.tobytes()))

        # --- Advance reference ---
        if time.time() - last_ref_time >= frame_delay:
            last_ref_time = time.time()
            ref_idx = (ref_idx + 1) % len(reference_sequence)

        time.sleep(0.01)

except KeyboardInterrupt:
    print("\nStopped by user")
finally:
    cap.release()
    pose.close()
    ipydisplay.clear_output()
    print("Cleanup complete")

Cleanup complete
